In [2]:
import requests
import json
import re
import sys
from bs4 import BeautifulSoup
import pandas as pd

soup = BeautifulSoup('html.parser')
soup

<html><body><p>html.parser</p></body></html>

In [4]:
# 서울시 25개 구의 법정동 코드 앞 5자리
SEOUL_DISTRICTS = {
    "강남구": "11680",
    "강동구": "11740",
    "강북구": "11305",
    "강서구": "11500",
    "관악구": "11620",
    "광진구": "11215",
    "구로구": "11530",
    "금천구": "11545",
    "노원구": "11350",
    "도봉구": "11320",
    "동대문구": "11230",
    "동작구": "11590",
    "마포구": "11440",
    "서대문구": "11410",
    "서초구": "11650",
    "성동구": "11200",
    "성북구": "11290",
    "송파구": "11710",
    "양천구": "11470",
    "영등포구": "11560",
    "용산구": "11170",
    "은평구": "11380",
    "종로구": "11110",
    "중구": "11140",
    "중랑구": "11260"
}


In [83]:
import time
import requests
import pandas as pd
import xml.etree.ElementTree as ET

# API 기본 설정
BASE_URL = "http://apis.data.go.kr/1613000/RTMSDataSvcAptTradeDev/getRTMSDataSvcAptTradeDev"
SERVICE_KEY = "YPxXvm/3jO5hggkbdJFWnhh8IC3VdMlkRvZHh1pkG8eZxXw12ymm4OAW10urfUVN++TzkurcrZ5Os3Gr9P8Kwg=="


# 데이터를 저장할 리스트
all_real_estate_data = []

# 크롤링할 기간 설정
start_year = 2020
start_month = 1
end_year = 2025
end_month = 2

current_year = start_year
current_month = start_month

# ⏳ 월별 반복문 실행
while (current_year < end_year) or (current_year == end_year and current_month <= end_month):
    deal_ymd = f"{current_year}{current_month:02}"
    print(f"\n⏳ {deal_ymd} 데이터 수집 시작...")

    for gu, code in SEOUL_DISTRICTS.items():
        print(f"📌 {gu} ({code}) 데이터 요청 중...")

        params = {
            "LAWD_CD": code,
            "DEAL_YMD": deal_ymd,
            "serviceKey": SERVICE_KEY,
            "pageNo": 1,
            "numOfRows": 999
        }

        response = requests.get(BASE_URL, params=params)
        response.encoding = "utf-8"

        if response.status_code == 200:
            root = ET.fromstring(response.text)

            items = root.findall(".//item")
            if not items:
                print(f"⚠️ {gu}에 거래 데이터가 없습니다.")
                continue

            # XML 데이터 -> 리스트 변환
            for item in items:
                data = {
                    'aptDong': item.find('aptDong').text if item.find('aptDong') is not None else '',
                    'aptNm': item.find('aptNm').text if item.find('aptNm') is not None else '',
                    'aptSeq': item.find('aptSeq').text if item.find('aptSeq') is not None else '',
                    'bonbun': item.find('bonbun').text if item.find('bonbun') is not None else '',
                    'bubun': item.find('bubun').text if item.find('bubun') is not None else '',
                    'buildYear': item.find('buildYear').text if item.find('buildYear') is not None else '',
                    'buyerGbn': item.find('buyerGbn').text if item.find('buyerGbn') is not None else '',
                    'cdealDay': item.find('cdealDay').text if item.find('cdealDay') is not None else '',
                    'cdealType': item.find('cdealType').text if item.find('cdealType') is not None else '',
                    'dealAmount': item.find('dealAmount').text if item.find('dealAmount') is not None else '',
                    'dealDay': item.find('dealDay').text if item.find('dealDay') is not None else '',
                    'dealMonth': item.find('dealMonth').text if item.find('dealMonth') is not None else '',
                    'dealYear': item.find('dealYear').text if item.find('dealYear') is not None else '',
                    'dealingGbn': item.find('dealingGbn').text if item.find('dealingGbn') is not None else '',
                    'estateAgentSggNm': item.find('estateAgentSggNm').text if item.find('estateAgentSggNm') is not None else '',
                    'excluUseAr': item.find('excluUseAr').text if item.find('excluUseAr') is not None else '',
                    'floor': item.find('floor').text if item.find('floor') is not None else '',
                    'jibun': item.find('jibun').text if item.find('jibun') is not None else '',
                    'landCd': item.find('landCd').text if item.find('landCd') is not None else '',
                    'landLeaseholdGbn': item.find('landLeaseholdGbn').text if item.find('landLeaseholdGbn') is not None else '',
                    'rgstDate': item.find('rgstDate').text if item.find('rgstDate') is not None else '',
                    'roadNm': item.find('roadNm').text if item.find('roadNm') is not None else '',
                    'roadNmBonbun': item.find('roadNmBonbun').text if item.find('roadNmBonbun') is not None else '',
                    'roadNmBubun': item.find('roadNmBubun').text if item.find('roadNmBubun') is not None else '',
                    'roadNmCd': item.find('roadNmCd').text if item.find('roadNmCd') is not None else '',
                    'roadNmSeq': item.find('roadNmSeq').text if item.find('roadNmSeq') is not None else '',
                    'roadNmSggCd': item.find('roadNmSggCd').text if item.find('roadNmSggCd') is not None else '',
                    'roadNmbCd': item.find('roadNmbCd').text if item.find('roadNmbCd') is not None else '',
                    'sggCd': item.find('sggCd').text if item.find('sggCd') is not None else '',
                    'slerGbn': item.find('slerGbn').text if item.find('slerGbn') is not None else '',
                    'umdCd': item.find('umdCd').text if item.find('umdCd') is not None else '',
                    'umdNm': item.find('umdNm').text if item.find('umdNm') is not None else ''
                }
                all_real_estate_data.append(data)

        else:
            print(f"❌ {gu} 응답 오류 ({response.status_code})")

    # ✅ 다음 달로 이동
    if current_month == 12:
        current_month = 1
        current_year += 1
    else:
        current_month += 1

# 📦 모든 데이터 DataFrame 변환
df_seoul_real_estate_data = pd.DataFrame(all_real_estate_data)

# ✅ 최종 데이터 확인
print("\n📊 최종 데이터 개수:", len(df_seoul_real_estate_data))
print(df_seoul_real_estate_data.head())


⏳ 202001 데이터 수집 시작...
📌 강남구 (11680) 데이터 요청 중...
📌 강동구 (11740) 데이터 요청 중...
📌 강북구 (11305) 데이터 요청 중...
📌 강서구 (11500) 데이터 요청 중...
📌 관악구 (11620) 데이터 요청 중...
📌 광진구 (11215) 데이터 요청 중...
📌 구로구 (11530) 데이터 요청 중...
📌 금천구 (11545) 데이터 요청 중...
📌 노원구 (11350) 데이터 요청 중...
📌 도봉구 (11320) 데이터 요청 중...
📌 동대문구 (11230) 데이터 요청 중...
📌 동작구 (11590) 데이터 요청 중...
📌 마포구 (11440) 데이터 요청 중...
📌 서대문구 (11410) 데이터 요청 중...
📌 서초구 (11650) 데이터 요청 중...
📌 성동구 (11200) 데이터 요청 중...
📌 성북구 (11290) 데이터 요청 중...
📌 송파구 (11710) 데이터 요청 중...
📌 양천구 (11470) 데이터 요청 중...
📌 영등포구 (11560) 데이터 요청 중...
📌 용산구 (11170) 데이터 요청 중...
📌 은평구 (11380) 데이터 요청 중...
📌 종로구 (11110) 데이터 요청 중...
📌 중구 (11140) 데이터 요청 중...
📌 중랑구 (11260) 데이터 요청 중...

⏳ 202002 데이터 수집 시작...
📌 강남구 (11680) 데이터 요청 중...
📌 강동구 (11740) 데이터 요청 중...
📌 강북구 (11305) 데이터 요청 중...
📌 강서구 (11500) 데이터 요청 중...
📌 관악구 (11620) 데이터 요청 중...
📌 광진구 (11215) 데이터 요청 중...
📌 구로구 (11530) 데이터 요청 중...
📌 금천구 (11545) 데이터 요청 중...
📌 노원구 (11350) 데이터 요청 중...
📌 도봉구 (11320) 데이터 요청 중...
📌 동대문구 (11230) 데이터 요청 중...
📌 동작구 (11590) 데

In [50]:
df_seoul_real_estate_data

,aptDong,aptNm,aptSeq,bonbun,bubun,buildYear,buyerGbn,cdealDay,cdealType,dealAmount,...,roadNmBonbun,roadNmBubun,roadNmCd,roadNmSeq,roadNmSggCd,roadNmbCd,sggCd,slerGbn,umdCd,umdNm
0,106,개포래미안포레스트,11680-4929,1282,0000,2020,개인,,,"286,000",...,00264,00000,3122001,01,11680,,11680,개인,10300,개포동
1,,개포우성1,11680-372,0503,0000,1983,개인,,,"420,000",...,00120,00000,3122006,00,11680,,11680,개인,10600,대치동
2,,삼성풍림2차아파트,11680-192,0104,0000,1998,개인,,,"165,000",...,00012,00007,4166326,01,11680,0,11680,개인,10500,삼성동
3,,개포래미안포레스트,11680-4929,1282,0000,2020,개인,25.03.01,O,"286,000",...,00264,00000,3122001,01,11680,,11680,개인,10300,개포동
4,,삼성,11680-330,0161,0000,1994,개인,,,"212,000",...,00014,00000,4166100,01,11680,0,11680,개인,11800,도곡동
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8314,,두산5,11260-3753,1522,0001,2000,개인,,,"63,000",...,00090,00000,4118361,01,11260,0,11260,개인,10100,면목동
8315,,사가정센트럴아이파크,11260-4104,1545,0000,2020,개인,,,"119,800",...,00040,00000,4118135,01,11260,0,11260,개인,10100,면목동
8316,,신도브래뉴,11260-149,0169,0010,2004,개인,,,"48,000",...,00946,00000,3000001,04,11260,0,11260,개인,10400,묵동
8317,,브라운스톤태릉,11260-108,0387,0000,2004,개인,,,"87,800",...,00059,00000,4118047,01,11260,0,11260,개인,10400,묵동


In [84]:
real_estate_schema = {
    "sggCd": "법정동시군구코드",
    "umdCd": "법정동읍면동코드",
    "landCd": "법정동지번코드",
    "bonbun": "법정동본번코드",
    "bubun": "법정동부번코드",
    "roadNm": "도로명",
    "roadNmSggCd": "도로명시군구코드",
    "roadNmCd": "도로명코드",
    "roadNmSeq": "도로명일련번호코드",
    "roadNmbCd": "도로명지상지하코드",
    "roadNmBonbun": "도로명건물본번호코드",
    "roadNmBubun": "도로명건물부번호코드",
    "umdNm": "법정동",
    "aptNm": "아파트명",
    "jibun": "지번",
    "excluUseAr": "전용면적",
    "dealYear": "계약년도",
    "dealMonth": "계약월",
    "dealDay": "계약일",
    "dealAmount": "거래금액(만원)",
    "floor": "층",
    "buildYear": "건축년도",
    "aptSeq": "단지 일련번호",
    "cdealType": "해제여부",
    "cdealDay": "해제사유발생일",
    "dealingGbn": "거래유형(중개 및 직거래 여부)",
    "estateAgentSggNm": "중개사소재지(시군구 단위)",
    "rgstDate": "등기일자",
    "aptDong": "아파트 동명",
    "slerGbn": "거래주체정보_매도자(개인/법인/공공기관/기타)",
    "buyerGbn": "거래주체정보_매수자(개인/법인/공공기관/기타)",
    "landLeaseholdGbn": "토지임대부 아파트 여부"
}


In [85]:
df_seoul_real_estate_data.rename(columns=real_estate_schema, inplace=True)
df_seoul_real_estate_data.head()

,아파트 동명,아파트명,단지 일련번호,법정동본번코드,법정동부번코드,건축년도,거래주체정보_매수자(개인/법인/공공기관/기타),해제사유발생일,해제여부,거래금액(만원),...,도로명건물본번호코드,도로명건물부번호코드,도로명코드,도로명일련번호코드,도로명시군구코드,도로명지상지하코드,법정동시군구코드,거래주체정보_매도자(개인/법인/공공기관/기타),법정동읍면동코드,법정동
0,,"현대1차(12,13,21,22,31,32,33동)",11680-415,0369,0001,1976,,,,"470,000",...,00071,00000,4166514,01,11680,0,11680,,11000,압구정동
1,,논현파라곤,11680-3726,0241,0001,2007,,,,"150,000",...,00338,00000,3122011,03,11680,0,11680,,10800,논현동
2,,타워팰리스2,11680-365,0467,0017,2003,,,,"310,000",...,00057,00000,4166604,01,11680,0,11680,,11800,도곡동
3,,청담스위트,11680-4364,0034,0013,2015,,,,"23,000",...,00013,00000,4166818,00,11680,0,11680,,10400,청담동
4,,삼성펠리스,11680-3582,0129,0005,2005,,,,"161,000",...,00031,00000,4166758,01,11680,0,11680,,10500,삼성동


In [86]:
df_seoul_real_estate_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 239853 entries, 0 to 239852
Data columns (total 32 columns):
 #   Column                     Non-Null Count   Dtype 
---  ------                     --------------   ----- 
 0   아파트 동명                     239853 non-null  object
 1   아파트명                       239853 non-null  object
 2   단지 일련번호                    239853 non-null  object
 3   법정동본번코드                    239853 non-null  object
 4   법정동부번코드                    239853 non-null  object
 5   건축년도                       239853 non-null  object
 6   거래주체정보_매수자(개인/법인/공공기관/기타)  239853 non-null  object
 7   해제사유발생일                    239853 non-null  object
 8   해제여부                       239853 non-null  object
 9   거래금액(만원)                   239853 non-null  object
 10  계약일                        239853 non-null  object
 11  계약월                        239853 non-null  object
 12  계약년도                       239853 non-null  object
 13  거래유형(중개 및 직거래 여부)          239853 non-null  

In [87]:
# 칼럼 데이터타입 변환환
df_seoul_real_estate_data['거래금액(만원)'] = df_seoul_real_estate_data['거래금액(만원)'].str.replace(",", "").astype(int)
df_seoul_real_estate_data['계약일'] = df_seoul_real_estate_data['계약일'].str.replace(",", "").astype(int)
df_seoul_real_estate_data['계약월'] = df_seoul_real_estate_data['계약월'].str.replace(",", "").astype(int)
df_seoul_real_estate_data['계약년도'] = df_seoul_real_estate_data['계약년도'].str.replace(",", "").astype(int)

In [88]:
# 전용면적 칼럼을 숫자형으로 변환
df_seoul_real_estate_data['전용면적'] = pd.to_numeric(df_seoul_real_estate_data['전용면적'], errors='coerce')

# 소수점 이하 버리기 (truncate 방식)
df_seoul_real_estate_data['전용면적'] = df_seoul_real_estate_data['전용면적'].apply(np.trunc).astype(int)
df_seoul_real_estate_data['전용면적']



0         196
1         120
2         164
3          13
4         174
         ... 
239848     59
239849     84
239850     51
239851     83
239852     84
Name: 전용면적, Length: 239853, dtype: int64

In [89]:
df_seoul_real_estate_data['평당가격'] = round(df_seoul_real_estate_data['거래금액(만원)'] / df_seoul_real_estate_data['전용면적'] , 2)
df_seoul_real_estate_data

,아파트 동명,아파트명,단지 일련번호,법정동본번코드,법정동부번코드,건축년도,거래주체정보_매수자(개인/법인/공공기관/기타),해제사유발생일,해제여부,거래금액(만원),...,도로명건물부번호코드,도로명코드,도로명일련번호코드,도로명시군구코드,도로명지상지하코드,법정동시군구코드,거래주체정보_매도자(개인/법인/공공기관/기타),법정동읍면동코드,법정동,평당가격
0,,"현대1차(12,13,21,22,31,32,33동)",11680-415,0369,0001,1976,,,,470000,...,00000,4166514,01,11680,0,11680,,11000,압구정동,2397.96
1,,논현파라곤,11680-3726,0241,0001,2007,,,,150000,...,00000,3122011,03,11680,0,11680,,10800,논현동,1250.00
2,,타워팰리스2,11680-365,0467,0017,2003,,,,310000,...,00000,4166604,01,11680,0,11680,,11800,도곡동,1890.24
3,,청담스위트,11680-4364,0034,0013,2015,,,,23000,...,00000,4166818,00,11680,0,11680,,10400,청담동,1769.23
4,,삼성펠리스,11680-3582,0129,0005,2005,,,,161000,...,00000,4166758,01,11680,0,11680,,10500,삼성동,925.29
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
239848,,두산5,11260-3753,1522,0001,2000,개인,,,63000,...,00000,4118361,01,11260,0,11260,개인,10100,면목동,1067.80
239849,,사가정센트럴아이파크,11260-4104,1545,0000,2020,개인,,,119800,...,00000,4118135,01,11260,0,11260,개인,10100,면목동,1426.19
239850,,신도브래뉴,11260-149,0169,0010,2004,개인,,,48000,...,00000,3000001,04,11260,0,11260,개인,10400,묵동,941.18
239851,,브라운스톤태릉,11260-108,0387,0000,2004,개인,,,87800,...,00000,4118047,01,11260,0,11260,개인,10400,묵동,1057.83


In [ ]:
# 평당가격을 평균 값으로 그룹화
grouped_seoul = df_seoul_real_estate_data.groupby(["법정동", "계약년도", "계약월"], as_index=False).agg({"평당가격": "mean"})
grouped_seoul

,법정동,계약년도,계약월,평당가격
0,가락동,2020,1,1161.586500
1,가락동,2020,2,1305.935758
2,가락동,2020,3,1099.383636
3,가락동,2020,4,1110.374545
4,가락동,2020,5,1214.206923
...,...,...,...,...
14649,흥인동,2024,10,1031.135000
14650,흥인동,2024,11,893.750000
14651,흥인동,2024,12,1208.120000
14652,흥인동,2025,1,1105.435000


In [91]:
# 지역=서울 칼럼 추가
grouped_seoul['지역'] = '서울'
grouped_seoul

,법정동,계약년도,계약월,평당가격,지역
0,가락동,2020,1,1161.586500,서울
1,가락동,2020,2,1305.935758,서울
2,가락동,2020,3,1099.383636,서울
3,가락동,2020,4,1110.374545,서울
4,가락동,2020,5,1214.206923,서울
...,...,...,...,...,...
14649,흥인동,2024,10,1031.135000,서울
14650,흥인동,2024,11,893.750000,서울
14651,흥인동,2024,12,1208.120000,서울
14652,흥인동,2025,1,1105.435000,서울


In [92]:
# 데이터 저장
grouped_seoul.to_csv("seoul.csv", index=False)